In [3]:
%load_ext autoreload
%autoreload 2
from typing import *

In [4]:
from utils import load_input

# Part 1

You land in an airport surrounded by dense forest. As you walk to your high-speed train, the Elves at the Mythical Information Bureau contact you again. They think their satellite has collected an image of a sea monster! Unfortunately, the connection to the satellite is having problems, and many of the messages sent back from the satellite have been corrupted.

They sent you a list of the rules valid messages should obey and a list of received messages they've collected so far (your puzzle input).

The rules for valid messages (the top part of your puzzle input) are numbered and build upon each other. For example:
```
0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"
```
Some rules, like 3: "b", simply match a single character (in this case, b).

The remaining rules list the sub-rules that must be followed; for example, the rule 0: 1 2 means that to match rule 0, the text being checked must match rule 1, and the text after the part that matched rule 1 must then match rule 2.

Some of the rules have multiple lists of sub-rules separated by a pipe (|). This means that at least one list of sub-rules must match. (The ones that match might be different each time the rule is encountered.) For example, the rule 2: 1 3 | 3 1 means that to match rule 2, the text being checked must match rule 1 followed by rule 3 or it must match rule 3 followed by rule 1.

Fortunately, there are no loops in the rules, so the list of possible matches will be finite. Since rule 1 matches a and rule 3 matches b, rule 2 matches either ab or ba. Therefore, rule 0 matches aab or aba.

Here's a more interesting example:
```
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"
```
Here, because rule 4 matches a and rule 5 matches b, rule 2 matches two letters that are the same (aa or bb), and rule 3 matches two letters that are different (ab or ba).

Since rule 1 matches rules 2 and 3 once each in either order, it must match two pairs of letters, one pair with matching letters and one pair with different letters. This leaves eight possibilities: aaab, aaba, bbab, bbba, abaa, abbb, baaa, or babb.

Rule 0, therefore, matches a (rule 4), then any of the eight options from rule 1, then b (rule 5): aaaabb, aaabab, abbabb, abbbab, aabaab, aabbbb, abaaab, or ababbb.

The received messages (the bottom part of your puzzle input) need to be checked against the rules so you can determine which are valid and which are corrupted. Including the rules and the messages together, this might look like:
```
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb
```
Your goal is to determine the number of messages that completely match rule 0. In the above example, ababbb and abbbab match, but bababa, aaabbb, and aaaabbb do not, producing the answer 2. The whole message must match all of rule 0; there can't be extra unmatched characters in the message. (For example, aaaabbb might appear to match rule 0 above, but it has an extra unmatched b on the end.)

How many messages completely match rule 0?



In [5]:
data = load_input(19, splitlines=False)
rules_raw, messages_raw = data.split("\n\n")

In [6]:
messages = messages_raw.split()

In [7]:
d = {
    0: [[4, 1, 5]],
    1: [[2, 3], [3, 2]],
    2: [[4, 4], [5, 5]],
    3: [[4, 5], [5, 4]],
    4: "a",
    5: "b"
}

In [8]:
import functools

def resolve_complex_key(d, k):
    @functools.lru_cache(maxsize=None)
    def _resolver(k):
        out = set()
        for vs in d[k]:
            possibility = [""]
            for v in vs:
                if isinstance(v, str):
                    possibility = set(v+p for p in possibility)
                else:
                    res = _resolver(v)
                    if isinstance(res, str):
                        possibility = set(p+res for p in possibility)
                    else:
                        possibility = set(l+r for l in possibility for r in res)
            out.update(possibility)
        return out
    out =  _resolver(k)
    print(_resolver.cache_info())
    return out


In [9]:
test_valid = resolve_complex_key(d, 0)

CacheInfo(hits=10, misses=6, maxsize=None, currsize=6)


In [10]:
test_messages = """ababbb
bababa
abbbab
aaabbb
aaaabbb
""".split()

In [11]:
len(list(filter(lambda x: x in test_valid, test_messages)))

2

In [12]:
import string 
def parse_rule(s: str):
    if len(s) == 3 and s[1] in string.ascii_lowercase:
        return s[1]
    else:
        parts = s.split(" | ")
        return [list(map(int, p.split())) for p in parts]

In [13]:
rules = {int(k): parse_rule(v) for k,v in [r.split(": ") for r in rules_raw.split("\n")]}

In [14]:
possibilities = resolve_complex_key(rules, 0)

CacheInfo(hits=370, misses=136, maxsize=None, currsize=136)


In [15]:
len(list(filter(lambda x: x in possibilities, messages)))

178

# Part 2

As you look over the list of messages, you realize your matching rules aren't quite right. To fix them, completely replace rules 8: 42 and 11: 42 31 with the following:

```
8: 42 | 42 8
11: 42 31 | 42 11 31
```
This small change has a big impact: now, the rules do contain loops, and the list of messages they could hypothetically match is infinite. You'll need to determine how these changes affect which messages are valid.

Fortunately, many of the rules are unaffected by this change; it might help to start by looking at which rules always match the same set of values and how those rules (especially rules 42 and 31) are used by the new versions of rules 8 and 11.

(Remember, you only need to handle the rules you have; building a solution that could handle any hypothetical combination of rules would be significantly more difficult.)

For example:

```
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
```

Without updating rules 8 and 11, these rules only match three messages: bbabbbbaabaabba, ababaaaaaabaaab, and ababaaaaabbbaba.

However, after updating rules 8 and 11, a total of 12 messages match:
```
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
```
After updating rules 8 and 11, how many messages completely match rule 0?



In [25]:
rules_2 = {int(k): parse_rule(v) for k,v in [r.split(": ") for r in rules_raw.split("\n")]}
rules_2[8] = [[42], [42, 8]]
rules_2[11] = [[42, 31], [42, 11, 31]]

We have 178 from the original list that pass the filter, and will pass this one. Care about which messages pass new rules

In [29]:
unmatched = list(filter(lambda x: x not in possibilities, messages))

- New rule for `8`: Any number of repititions of text from `42`
- New rule for `11`: Any number of repititions of `42` followed by `31` once or twice

Given `0: 8 11`,  the general rule for `0` is at least two of of `42` followed by `31` one or two times.

In [102]:
possibilities_42 = resolve_complex_key(rules_2, 42)
possibilities_31 = resolve_complex_key(rules_2, 31)

CacheInfo(hits=213, misses=78, maxsize=None, currsize=78)
CacheInfo(hits=207, misses=78, maxsize=None, currsize=78)


In [103]:
import re

In [104]:
match_group_42 = "("+"|".join(possibilities_42)+")"
match_group_31 = "("+"|".join(possibilities_31)+")"
pattern = re.compile(match_group_42+"{2,}"+match_group_31+"{1,3}") # Note - I have no clue why 3 is the match here

In [105]:
len(list(filter(lambda x: re.fullmatch(pattern, x), messages)))

346